<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3 Interlude: Chisel Standard Library
**Prev: [Generators: Collections](3.2_collections.ipynb)**<br>
**Next: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**

## Motivation
Chisel is all about re-use, so it only makes sense to provide a standard library of interfaces (encouraging interoperability of RTL) and generators for commonly-used hardware blocks.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /Users/dan/dev/dan-fritchman/chisel-bootcamp/Main.sc

Compiling /Users/dan/dev/dan-fritchman/chisel-bootcamp/Main.sc #2

path: String = "/Users/dan/dev/dan-fritchman/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# The Cheatsheet
The [Chisel3 cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf) contains a summary of all the major hardware construction APIs, including some of the standard library utilities that we'll introduce below.

# Decoupled: A Standard Ready-Valid Interface
One of the commonly used interfaces provided by Chisel is `DecoupledIO`, providing a ready-valid interface for transferring data. The idea is that the source drives the `bits` signal with the data to be transferred and the `valid` signal when there is data to be transferred. The sink drives the `ready` signal when it is ready to accept data, and data is considered transferred when both `ready` and `valid` are asserted on a cycle.

This provides a flow control mechanism in both directions for data transfer, including a backpressure mechanism.

Note: `ready` and `valid` should not be combinationally coupled, otherwise this may result in unsynthesizable combinational loops. `ready` should only be dependent on whether the sink is able to receive data, and `valid` should only be dependent on whether the source has data. Only after the transaction (on the next clock cycle) should the values update.

Any Chisel data can be wrapped in a `DecoupledIO` (used as the `bits` field) as follows:

```scala
val myChiselData = UInt(8.W)
// or any Chisel data type, such as Bool(), SInt(...), or even custom Bundles
val myDecoupled = Decoupled(myChiselData)
```

The above creates a new `DecoupledIO` Bundle with fields
- `valid`: Output(Bool)
- `ready`: Input(Bool)
- `bits`: Output(UInt(8.W))
___

The rest of the section will be structured somewhat differently from the ones before: instead of giving you coding exercises, we're going to give some code examples and testcases that print the circuit state. Try to predict what will be printed before just running the tests.

## Queues

`Queue` creates a FIFO (first-in, first-out) queue with Decoupled interfaces on both sides, allowing backpressure. Both the data type and number of elements are configurable.

In [3]:
Driver(() => new Module {
    // Example circuit using a Queue
    val io = IO(new Bundle {
      val in = Flipped(Decoupled(UInt(8.W)))
      val out = Decoupled(UInt(8.W))
    })
    val queue = Queue(io.in, 2)  // 2-element queue
    io.out <> queue
  }) { c => new PeekPokeTester(c) {
    // Example testsequence showing the use and behavior of Queue
    poke(c.io.out.ready, 0)
    poke(c.io.in.valid, 1)  // Enqueue an element
    poke(c.io.in.bits, 42)
    println(s"Starting:")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 1)  // Enqueue another element
    poke(c.io.in.bits, 43)
    // What do you think io.out.valid and io.out.bits will be?
    println(s"After first enqueue:")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 1)  // Read a element, attempt to enqueue
    poke(c.io.in.bits, 44)
    poke(c.io.out.ready, 1)
    // What do you think io.in.ready will be, and will this enqueue succeed, and what will be read?
    println(s"On first read:")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 0)  // Read elements out
    poke(c.io.out.ready, 1)
    // What do you think will be read here?
    println(s"On second read:")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    // Will a third read produce anything?
    println(s"On third read:")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
} }

[info] [0.002] Elaborating design...
[info] [0.996] Done elaborating.
Total FIRRTL Compile Time: 639.1 ms
Total FIRRTL Compile Time: 130.6 ms
End of dependency graph
Circuit state created
[info] [0.004] SEED 1575033418901
[info] [0.021] Starting:
[info] [0.037] 	io.in: ready=1
[info] [0.039] 	io.out: valid=0, bits=173
[info] [0.064] After first enqueue:
[info] [0.065] 	io.in: ready=1
[info] [0.074] 	io.out: valid=1, bits=42
[info] [0.084] On first read:
[info] [0.090] 	io.in: ready=0
[info] [0.096] 	io.out: valid=1, bits=42
[info] [0.115] On second read:
[info] [0.118] 	io.in: ready=1
[info] [0.126] 	io.out: valid=1, bits=43
[info] [0.128] On third read:
[info] [0.132] 	io.in: ready=1
[info] [0.134] 	io.out: valid=0, bits=42
test _1 Success: 0 tests passed in 10 cycles taking 0.175850 seconds
[info] [0.157] RAN 5 CYCLES PASSED


res2: Boolean = true

## Arbiters
Arbiters routes data from _n_ `DecoupledIO` sources to one `DecoupledIO` sink, given a prioritization.
There are two types included in Chisel:
- `Arbiter`: prioritizes lower-index producers
- `RRArbiter`: runs in round-robin order

Note that Arbiter routing is implemented in combinational logic.

The below example will demonstrate the use of the priority arbiter (which you will also implement in the next section):

In [4]:
Driver(() => new Module {
    // Example circuit using a priority arbiter
    val io = IO(new Bundle {
      val in = Flipped(Vec(2, Decoupled(UInt(8.W))))
      val out = Decoupled(UInt(8.W))
    })
    // Arbiter doesn't have a convenience constructor, so it's built like any Module
    val arbiter = Module(new Arbiter(UInt(8.W), 2))  // 2 to 1 Priority Arbiter
    arbiter.io.in <> io.in
    io.out <> arbiter.io.out
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in(0).valid, 0)
    poke(c.io.in(1).valid, 0)
    println(s"Start:")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(1).valid, 1)  // Valid input 1
    poke(c.io.in(1).bits, 42)
    // What do you think the output will be?
    println(s"valid input 1:")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(0).valid, 1)  // Valid inputs 0 and 1
    poke(c.io.in(0).bits, 43)
    // What do you think the output will be? Which inputs will be ready?
    println(s"valid inputs 0 and 1:")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(1).valid, 0)  // Valid input 0
    // What do you think the output will be?
    println(s"valid input 0:")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
} }

[info] [0.000] Elaborating design...
[info] [0.162] Done elaborating.
Total FIRRTL Compile Time: 109.5 ms
Total FIRRTL Compile Time: 106.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033549856
[info] [0.001] Start:
[info] [0.007] 	in(0).ready=1, in(1).ready=1
[info] [0.015] 	out.valid=0, out.bits=49
[info] [0.016] valid input 1:
[info] [0.034] 	in(0).ready=1, in(1).ready=1
[info] [0.040] 	out.valid=1, out.bits=42
[info] [0.041] valid inputs 0 and 1:
[info] [0.051] 	in(0).ready=1, in(1).ready=0
[info] [0.053] 	out.valid=1, out.bits=43
[info] [0.062] valid input 0:
[info] [0.064] 	in(0).ready=1, in(1).ready=0
[info] [0.066] 	out.valid=1, out.bits=43
test _1 Success: 0 tests passed in 5 cycles taking 0.076981 seconds
[info] [0.068] RAN 0 CYCLES PASSED


res3: Boolean = true

# Misc Function Blocks
Chisel Utils has some helpers that perform stateless functions.

## Bitwise Utilities
### PopCount
PopCount returns the number of high (1) bits in the input as a `UInt`.

### Reverse
Reverse returns the bit-reversed input.

In [5]:
Driver(() => new Module {
    // Example circuit using Reverse
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := PopCount(io.in)
  }) { c => new PeekPokeTester(c) {
    // Integer.parseInt is used create an Integer from a binary specification
    poke(c.io.in, Integer.parseInt("00000000", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("00001111", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("11001010", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("11111111", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
} }

[info] [0.000] Elaborating design...
[info] [0.080] Done elaborating.
Total FIRRTL Compile Time: 55.2 ms
Total FIRRTL Compile Time: 52.1 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1575033591311
[info] [0.017] in=0b0, out=0
[info] [0.031] in=0b1111, out=4
[info] [0.037] in=0b11001010, out=4
[info] [0.052] in=0b11111111, out=8
test _1 Success: 0 tests passed in 5 cycles taking 0.068548 seconds
[info] [0.073] RAN 0 CYCLES PASSED


res4: Boolean = true

In [6]:
Driver(() => new Module {
    // Example circuit using Reverse
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := Reverse(io.in)
  }) { c => new PeekPokeTester(c) {
    // Integer.parseInt is used create an Integer from a binary specification
    poke(c.io.in, Integer.parseInt("01010101", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, Integer.parseInt("00001111", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, Integer.parseInt("11110000", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, Integer.parseInt("11001010", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
} }

[info] [0.000] Elaborating design...
[info] [0.012] Done elaborating.
Total FIRRTL Compile Time: 81.0 ms
Total FIRRTL Compile Time: 61.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033611856
[info] [0.014] in=0b1010101, out=0b10101010
[info] [0.030] in=0b1111, out=0b11110000
[info] [0.037] in=0b11110000, out=0b1111
[info] [0.039] in=0b11001010, out=0b1010011
test _1 Success: 0 tests passed in 5 cycles taking 0.048383 seconds
[info] [0.041] RAN 0 CYCLES PASSED


res5: Boolean = true

## OneHot encoding utilities
OneHot is an encoding of integers where there is one wire for each value, and exactly one wire is high. This allows the efficient creation of some functions, for example muxes. However, behavior may be undefined if the one-wire-high condition is not held.

The below two functions provide conversion between binary (`UInt`) and OneHot encodings, and are inverses of each other:
- UInt to OneHot: `UIntToOH`
- OneHot to UInt: `OHToUInt`

In [7]:
Driver(() => new Module {
    // Example circuit using UIntToOH
    val io = IO(new Bundle {
      val in = Input(UInt(4.W))
      val out = Output(UInt(16.W))
    })
    io.out := UIntToOH(io.in)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")

    poke(c.io.in, 1)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, 8)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, 15)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")
} }

[info] [0.001] Elaborating design...
[info] [0.010] Done elaborating.
Total FIRRTL Compile Time: 38.5 ms
Total FIRRTL Compile Time: 43.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033650768
[info] [0.001] in=0, out=0b1
[info] [0.005] in=1, out=0b10
[info] [0.023] in=8, out=0b100000000
[info] [0.024] in=15, out=0b1000000000000000
test _1 Success: 0 tests passed in 5 cycles taking 0.030829 seconds
[info] [0.025] RAN 0 CYCLES PASSED


res6: Boolean = true

In [8]:
Driver(() => new Module {
    // Example circuit using OHToUInt
    val io = IO(new Bundle {
      val in = Input(UInt(16.W))
      val out = Output(UInt(4.W))
    })
    io.out := OHToUInt(io.in)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in, Integer.parseInt("0000 0000 0000 0001".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")

    poke(c.io.in, Integer.parseInt("0000 0000 1000 0000".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("1000 0000 0000 0001".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    // Some invalid inputs:
    // None high
    poke(c.io.in, Integer.parseInt("0000 0000 0000 0000".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    // Multiple high
    poke(c.io.in, Integer.parseInt("0001 0100 0010 0000".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
} }

[info] [0.000] Elaborating design...
[info] [0.013] Done elaborating.
Total FIRRTL Compile Time: 43.5 ms
Total FIRRTL Compile Time: 46.3 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033653625
[info] [0.006] in=0b1, out=0
[info] [0.017] in=0b10000000, out=7
[info] [0.018] in=0b1000000000000001, out=15
[info] [0.033] in=0b0, out=0
[info] [0.038] in=0b1010000100000, out=15
test _1 Success: 0 tests passed in 5 cycles taking 0.045846 seconds
[info] [0.040] RAN 0 CYCLES PASSED


res7: Boolean = true

## Muxes
These muxes take in a list of values with select signals, and output the value associated with the lowest-index select signal.

These can either take a list of (select: Bool, value: Data) tuples, or corresponding lists of selects and values as arguments. For simplicity, the examples below only demonstrate the second form.

### Priority Mux
A `PriorityMux` outputs the value associated with the lowest-index asserted select signal.

### OneHot Mux
An `Mux1H` provides an efficient implementation when it is guaranteed that exactly one of the select signals will be high. Behavior is undefined if the assumption is not true.

In [9]:
Driver(() => new Module {
    // Example circuit using PriorityMux
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := PriorityMux(io.in_sels, io.in_bits)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in_bits(0), 10)
    poke(c.io.in_bits(1), 20)
  
    // Select higher index only
    poke(c.io.in_sels(0), 0)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select both - arbitration needed
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select lower index only
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 0)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
} }

[info] [0.000] Elaborating design...
[info] [0.010] Done elaborating.
Total FIRRTL Compile Time: 28.1 ms
Total FIRRTL Compile Time: 53.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033698499
[info] [0.003] in_sels=Vector(0, 1), out=20
[info] [0.004] in_sels=Vector(1, 1), out=10
[info] [0.005] in_sels=Vector(1, 0), out=10
test _1 Success: 0 tests passed in 5 cycles taking 0.011827 seconds
[info] [0.011] RAN 0 CYCLES PASSED


res8: Boolean = true

In [10]:
Driver(() => new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := Mux1H(io.in_sels, io.in_bits)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in_bits(0), 10)
    poke(c.io.in_bits(1), 20)
  
    // Select index 1
    poke(c.io.in_sels(0), 0)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select index 0
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 0)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select none (invalid)
    poke(c.io.in_sels(0), 0)
    poke(c.io.in_sels(1), 0)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select both (invalid)
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
} }

[info] [0.000] Elaborating design...
[info] [0.080] Done elaborating.
Total FIRRTL Compile Time: 31.3 ms
Total FIRRTL Compile Time: 40.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033715131
[info] [0.002] in_sels=Vector(0, 1), out=20
[info] [0.008] in_sels=Vector(1, 0), out=10
[info] [0.011] in_sels=Vector(0, 0), out=0
[info] [0.019] in_sels=Vector(1, 1), out=30
test _1 Success: 0 tests passed in 5 cycles taking 0.027039 seconds
[info] [0.035] RAN 0 CYCLES PASSED


res9: Boolean = true

## Counter
`Counter` is a counter that can be incremented once every cycle, up to some specified limit, at which point it overflows. Note that it is **not** a Module, and its value is accessible.

In [11]:
Driver(() => new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val count = Input(Bool())
      val out = Output(UInt(2.W))
    })
    val counter = Counter(3)  // 3-count Counter (outputs range [0...2])
    when(io.count) {
      counter.inc()
    }
    io.out := counter.value
  }) { c => new PeekPokeTester(c) {
    poke(c.io.count, 1)
    println(s"start: counter value=${peek(c.io.out)}")
  
    step(1)
    println(s"step 1: counter value=${peek(c.io.out)}")
  
    step(1)
    println(s"step 2: counter value=${peek(c.io.out)}")
  
    poke(c.io.count, 0)
    step(1)
    println(s"step without increment: counter value=${peek(c.io.out)}")
  
    poke(c.io.count, 1)
    step(1)
    println(s"step again: counter value=${peek(c.io.out)}")
} }

[info] [0.000] Elaborating design...
[info] [0.088] Done elaborating.
Total FIRRTL Compile Time: 31.0 ms
Total FIRRTL Compile Time: 31.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033741648
[info] [0.004] start: counter value=0
[info] [0.007] step 1: counter value=1
[info] [0.008] step 2: counter value=2
[info] [0.020] step without increment: counter value=2
[info] [0.022] step again: counter value=0
test _1 Success: 0 tests passed in 9 cycles taking 0.024461 seconds
[info] [0.024] RAN 4 CYCLES PASSED


res10: Boolean = true

---
# You're done!

[Return to the top.](#top)